In [16]:
import xarray as xr
import time, sys
st = time.time()
#xr.open_mfdataset('~/R/gebco/*.nc', parallel=True)
ds = xr.open_mfdataset(
    '../data_src/GEBCO_2023_sub_ice_topo.nc', chunks={0: 60, 1: 60}, 
    decode_cf=False, decode_times=False) #.load()
et = time.time()
print('Exe time: ', et-st, 'sec')
ds

Exe time:  0.012810468673706055 sec


<xarray.Dataset>
Dimensions:    (lon: 86400, lat: 43200)
Coordinates:
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
Data variables:
    elevation  (lat, lon) int16 dask.array<chunksize=(43200, 86400), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        The GEBCO_2020 Grid - a continuous terrain model for oceans...
    institution:  On behalf of the General Bathymetric Chart of the Oceans (G...
    source:       The GEBCO_2020 Grid is the latest global bathymetric produc...
    history:      Information on the development of the data set and the sour...
    references:   DOI: 10.5285/836f016a-33be-6ddc-e053-6c86abc0788e
    comment:      The data in the GEBCO_2020 Grid should not be used for navi...
    node_offset:  1.0

In [2]:
print(sys.getsizeof(ds))
ds['elevation'].nbytes/(1024 ** 3)

112


6.9522857666015625

In [8]:
import xarray as xr
import dask

# Open the GEBCO_2022 dataset
ds1 = xr.open_mfdataset('../data_src/GEBCO_2022_sub_ice_topo.nc', chunks={0: 60, 1: 60},
                        decode_cf=False, decode_times=False)

# Open the GEBCO_2023 dataset
ds2 = xr.open_mfdataset('../data_src/GEBCO_2023_sub_ice_topo.nc', chunks={0: 60, 1: 60},
                        decode_cf=False, decode_times=False)

# Inspect the structure and metadata
print(ds1)
print(ds2)


<xarray.Dataset>
Dimensions:    (lon: 86400, lat: 43200)
Coordinates:
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(43200, 86400), meta=np.ndarray>
Attributes: (12/36)
    title:                           The GEBCO_2022 Grid - a continuous terra...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:  

In [9]:
print(isinstance(ds1['elevation'].data, dask.array.Array))

True


In [ ]:
# Assuming ds1 uses Dask, get its chunk sizes
# chunk_sizes = {dim: ds1[dim].data.chunksize for dim in ds1.dims}

# Apply these chunk sizes to ds2
# ds2 = ds2.chunk(chunk_sizes)


In [13]:
print(ds1.attrs)
print("---------------")
print(ds2.attrs)
print("--------------- data_vars")
for var in ds1.data_vars:
    print(ds1[var].attrs)

print("--------------- data_vars in ds2")

for var in ds2.data_vars:
    print(ds2[var].attrs)


{'title': 'The GEBCO_2022 Grid - a continuous terrain model for oceans and land at 15 arc-second intervals', 'summary': 'The GEBCO_2022 Grid is a continuous, global terrain model for ocean and land with a spatial resolution of 15 arc seconds.The grid uses as a base-map Version 2.4 of the SRTM15+ data set (Tozer et al, 2019). This data set is a fusion of land topography with measured and estimated seafloor topography. It is augmented with gridded bathymetric data sets developed as part of the Nippon Foundation-GEBCO Seabed 2030 Project.', 'keywords': 'BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ELEVATION/DIGITAL TERRAIN MODELS', 'Conventions': 'CF-1.6, ACDD-1.3', 'id': 'DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c86abc0289c', 'naming_authority': 'https://dx.doi.org', 'history': 'Information on the development of the data set and the source data sets included in the grid can be found in the data set documentation available from https://www.gebco.net', 'source': 'The GEBCO_2022 Grid is the latest 

In [ ]:
# Example: Copying global attributes
# ds2.attrs = ds1.attrs

# Example: Copying variable-specific attributes
# for var in ds2.data_vars:
#    if var in ds1.data_vars:
#        ds2[var].attrs = ds1[var].attrs

In [14]:
chunk_sizes = {'lat': 60, 'lon': 60}  # Replace with the actual chunk sizes used for ds1

# Apply these chunk sizes to ds2
ds2 = ds2.chunk(chunk_sizes)
ds2.to_zarr('../data/GEBCO_2023_sub_ice_topo.zarr', mode='w')

In [43]:
st = time.time()
#larger chunk size
ds2 = xr.open_mfdataset(
    '../data_src/GEBCO_2023_sub_ice_topo.nc', chunks={0: 45, 1: 45}, #{0: 120, 1: 120}
    decode_cf=False, decode_times=False) #.load()
et = time.time()
print('larger-chunk read time: ', et-st, 'sec')
print(sys.getsizeof(ds2))
ds2['elevation'].nbytes/(1024 ** 3)

st = time.time()
#smaller chunk size
ds3 = xr.open_mfdataset(
    '../data_src/GEBCO_2023_sub_ice_topo.nc', chunks={0: 30, 1: 30}, 
    decode_cf=False, decode_times=False) #.load()
et = time.time()
print('smaller-chunk read time: ', et-st, 'sec')
print(sys.getsizeof(ds2))
ds2['elevation'].nbytes/(1024 ** 3)

larger-chunk read time:  0.007239341735839844 sec
112
smaller-chunk read time:  0.0065441131591796875 sec
112


6.9522857666015625

In [2]:
print(ds.sel(lat=23.326173, lon=123.978125, method='nearest')['elevation'].values)
print(ds.sel(lat=23.317670, lon=123.973958, method='nearest')['elevation'].values)
print(ds.sel(lat=21.336378, lon=123.003125, method='nearest')['elevation'].values)
print(ds.sel(lat=21.33, lon=123.0, method='nearest')['elevation'].values)
print(ds["elevation"].values[27200,72956]) #this is correct value (got depth = -3729)
print(ds["elevation"].values[27200,72957])
print(ds["elevation"].values[27200,72955])
print(ds["elevation"].values[27201,72956])
print(ds["elevation"].values[27199,72956]) #But we got -3813, this is: after subset got a latitude offset 1? why (in zprofile01.ipynb correct it)

-4008
-4311
-4805
-4770
-3752
-3739
-3761
-3691
-3840


In [6]:
#The following try different subsetting and index methods
st = time.time()
ds_s1 = ds.sel(lon=slice(105, 135), lat=slice(2, 35)) 
ds_s1.close() #try if it can close
et = time.time()
print('Slicing time for chunk-60: ', et-st, 'sec')

st = time.time()
ds_s2 = ds2.sel(lon=slice(105, 135), lat=slice(2, 35)) 
ds_s2.close() #try if it can close
et = time.time()
print('Slicing time for larger chunk: ', et-st, 'sec')

st = time.time()
ds_s3 = ds3.sel(lon=slice(105, 135), lat=slice(2, 35)) 
ds_s3.close() #try if it can close
et = time.time()
print('Exe time for smaller chunk: ', et-st, 'sec')

Slicing time for chunk-60:  0.006575822830200195 sec
Slicing time for larger chunk:  0.005826711654663086 sec
Exe time for smaller chunk:  0.0052683353424072266 sec


In [9]:
ds_s1['elevation'].nbytes/(1024 ** 3)

0.1062154769897461

In [14]:
#da1 = getattr(ds_s1, 'elevation')
#da1
#https://www.programcreek.com/python/example/123571/xarray.decode_cf
#da1.attrs['_Unsigned']='false'
ds_s1


<xarray.Dataset>
Dimensions:    (lon: 7200, lat: 7920)
Coordinates:
  * lon        (lon) float64 105.0 105.0 105.0 105.0 ... 135.0 135.0 135.0 135.0
  * lat        (lat) float64 2.002 2.006 2.01 2.015 ... 34.99 34.99 34.99 35.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(7920, 7200), meta=np.ndarray>
Attributes: (12/36)
    title:                           The GEBCO_2022 Grid - a continuous terra...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    node_offset:                     1.0

In [ ]:
#da1

In [7]:
st = time.time()
st1 = ds_s1.isel(lon=slice(1000, 1010), lat=slice(2000, 2010))
xt1 = st1['elevation'].values
et = time.time()
print('Exe time: ', et-st, 'sec')
print(xt1)

Exe time:  0.009853124618530273 sec
[[-121 -119 -116 -111 -106 -101  -94  -88  -82  -76]
 [-123 -121 -119 -115 -109 -103  -97  -91  -86  -79]
 [-126 -124 -122 -118 -113 -106 -100  -95  -88  -83]
 [-127 -127 -126 -123 -116 -110 -103  -97  -91  -86]
 [-127 -127 -128 -126 -118 -112 -106  -99  -94  -88]
 [-127 -127 -126 -125 -119 -113 -107 -102  -96  -91]
 [-127 -127 -126 -123 -118 -113 -108 -103  -99  -94]
 [-126 -125 -124 -122 -118 -113 -109 -104 -100  -96]
 [-125 -124 -123 -120 -118 -114 -110 -107 -102  -98]
 [-123 -123 -121 -120 -118 -114 -112 -108 -104 -100]]


In [8]:
import numpy as np
print(st1["elevation"].shape)
st = time.time()
pt1 = [st1["elevation"].values[3,4], st1["elevation"].values[1,2]]
et = time.time()
print('3.1 Get index of point by two subsetting time: ', et-st, 'sec')

st = time.time()
#pt2 = st1["elevation"].values[np.ix_([3,4],[1,2])]
idx_arr = np.array([[3,4],[1,2]])
print(idx_arr)
print(tuple(idx_arr.T))
#https://stackoverflow.com/questions/43034563/numpy-array-of-multiple-indices-replace-with-a-different-matrix
pt2=st1['elevation'].values[tuple(idx_arr.T)]
et = time.time()
print('3.2 Get index of point by numpy ix_ time: ', et-st, 'sec')
print(pt1)
print(pt2)

(10, 10)
3.1 Get index of point by two subsetting time:  0.004407644271850586 sec
[[3 4]
 [1 2]]
(array([3, 1]), array([4, 2]))
3.2 Get index of point by numpy ix_ time:  0.002244710922241211 sec
[-116, -119]
[-116 -119]


In [ ]:
#slow
x = xr.DataArray([-45,-44.95833,-44,-43.95822], dims=['longitude'])
y = xr.DataArray([35,35.004167,36,36.004168], dims=['latitude'])
st = time.time()
#ds_s2 = ds.sel(lon=x, lat=y, method='nearest')  
et = time.time()
#print('Exe time: ', et-st, 'sec')
#print(ds_s2)               
#print(ds_s2['elevation'].values)

In [ ]:
#https://stackoverflow.com/questions/58706834/angled-slice-in-xarray
startLat = 35.0
startLon = -45.0
endLat = 36.0
endLon = -43.0
n = 3600/15 #15 arc-second
# not work, need another dimension to project
#x = xr.DataArray(np.linspace(startLon,endLon,n), dims='z')
#y = xr.DataArray(np.linspace(startLat,endLat,n), dims='z')
#st = time.time()
#ds_s3 = ds.interp(lon=x, lat=y, method='linear')
#et = time.time()
#print('Exe time: ', et-st, 'sec')
#print(ds_s3)               


In [ ]:
import orjson
import json
st = time.time()
jt1 = orjson.dumps(xt1.tolist(), option=orjson.OPT_NAIVE_UTC | orjson.OPT_SERIALIZE_NUMPY)
et = time.time()
print('4.1 Convert JSON by orjson time: ', et-st, 'sec')
st = time.time()
jt2 = json.dumps(xt1.tolist())
et = time.time()
print('4.2 Convert JSON by json.dump time: ', et-st, 'sec')

print(jt1)
print(jt2)

In [ ]:
ds.close()

In [25]:
import zarr
import dask

In [17]:
#compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
dask.config.set(scheduler='single-threaded')
#da = getattr(ds, 'elevation')
#https://www.programcreek.com/python/example/123571/xarray.decode_cf
#da.attrs['_Unsigned']='false'
#ds3 #### try 0:30 chunk #### not work
#In GEBCO_2022 it works but in GEBCO_2023, it encounter ValueError: Codec does not support buffers of > 2147483647 bytes
# https://chat.openai.com/c/67eaa069-b863-4924-9f56-1a1ef98c8b79 
#ds.to_zarr('../data/GEBCO_2023_sub_ice_topo.zarr', mode='w',
#           #encoding={"elevation": {"_Unsigned": "false"}},
#           group='gebco') ##compute=False, , encoding={'gebbco': {'compressor': compressor}})


In [ ]:
##import xarray as xr
##import zarr
#from numcodecs import Zlib

## Open your dataset
#ds = xr.open_mfdataset('../data_src/GEBCO_2023_sub_ice_topo.nc', chunks={0: 60, 1: 60}, 
#                       decode_cf=False, decode_times=False)

## Specify the compressor
#compressor = Zlib(level=1)  # Level can be adjusted for compression speed vs ratio

## Update encoding for each variable in dataset
#encoding = {var: {'compressor': compressor} for var in ds.data_vars}

## Convert to Zarr
#ds.to_zarr('../data/GEBCO_2023_sub_ice_topo_compressor.zarr', mode='w', group='gebco', encoding=encoding)


In [ ]:
#import xarray as xr
#import zarr
from numcodecs import Zlib
#from numcodecs import Blosc


# Open your dataset from NetCDF
ds = xr.open_mfdataset('../data_src/GEBCO_2023_sub_ice_topo.nc', chunks={0: 60, 1: 60}, 
                       decode_cf=False, decode_times=False)

# Specify the compressor
compressor = Zlib(level=1)  # Level can be adjusted for compression speed vs ratio

# Define the Blosc compressor
# blosc_compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)

# Apply the Blosc compressor to all variables

# Convert to Zarr without specifying a group, and with the new encoding
# Update encoding for each variable in dataset
encoding = {var: {'compressor': compressor} for var in ds.data_vars}
#encoding = {var: {'compressor': blosc_compressor} for var in ds.data_vars}

# Convert to Zarr without specifying a group
ds.to_zarr('../data/GEBCO_2023_sub_ice_topo_compressor.zarr', mode='w', encoding=encoding)
#ds.to_zarr('../data/GEBCO_2023_sub_ice_topo.zarr', mode='w', encoding=encoding)


In [1]:
import xarray as xr
from numcodecs import Blosc
from dask.distributed import Client, progress

# Start a Dask distributed client for parallel processing and monitoring
client = Client('tcp://localhost:8786')

print(f"Dask Dashboard is available at {client.dashboard_link}")

# Open your dataset with specified chunk sizes
ds = xr.open_mfdataset('../data_src/GEBCO_2023_sub_ice_topo.nc', 
                       # chunks={'lat': 30, 'lon': 30}, #why it's too slow
                       chunks={0: 60, 1: 60},
                       decode_cf=False, decode_times=False)

# Define the Blosc compressor
blosc_compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0, max_buffer_size=2**40 - 1)

# Apply the Blosc compressor to all variables
encoding = {var: {'compressor': blosc_compressor} for var in ds.data_vars}

# Convert to Zarr
future = client.persist(ds.to_zarr('../data/GEBCO_2023_sub_ice_topo.zarr', mode='w', encoding=encoding))
progress(future) 

Dask Dashboard is available at http://localhost:8787/status


TypeError: __init__() got an unexpected keyword argument 'max_buffer_size'

In [15]:
st = time.time()
dz1 = xr.open_zarr(
    '../data/GEBCO_2023_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
dz1

NameError: name 'time' is not defined

In [7]:
st = time.time()
dz2 = xr.open_zarr(
    '../data/GEBCO_2023_sub_ice_topo_compressor.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
dz2

Exe time:  0.007412433624267578 sec


<xarray.Dataset>
Dimensions:    (lat: 43200, lon: 86400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Data variables:
    elevation  (lat, lon) int16 dask.array<chunksize=(43200, 86400), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    comment:      The data in the GEBCO_2020 Grid should not be used for navi...
    history:      Information on the development of the data set and the sour...
    institution:  On behalf of the General Bathymetric Chart of the Oceans (G...
    node_offset:  1.0
    references:   DOI: 10.5285/836f016a-33be-6ddc-e053-6c86abc0788e
    source:       The GEBCO_2020 Grid is the latest global bathymetric produc...
    title:        The GEBCO_2020 Grid - a continuous terrain model for oceans...

In [39]:
# import xarray as xr
from numcodecs import Blosc

# Open the existing Zarr dataset
ds_compress = xr.open_zarr('../data/GEBCO_2023_sub_ice_topo_compressor.zarr')

blosc_compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)

# Apply the Blosc compressor to all variables
encoding = {var: {'compressor': blosc_compressor} for var in ds.data_vars}

# Convert to Zarr without specifying a group, and with the new encoding
ds_compress.to_zarr('../data/GEBCO_2023_sub_ice_topo.zarr', mode='w', encoding=encoding)
           #encoding={"elevation": {"_Unsigned": "false"}},
           #group='gebco') 


ValueError: Codec does not support buffers of > 2147483647 bytes

In [18]:
st = time.time()
dz = xr.open_zarr(
    '../data/GEBCO_2023_sub_ice_topo.zarr', chunks='auto', #group='gebco', 
    decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
dz

Exe time:  0.008665800094604492 sec


<xarray.Dataset>
Dimensions:    (lat: 43200, lon: 86400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Data variables:
    elevation  (lat, lon) int16 dask.array<chunksize=(60, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    comment:      The data in the GEBCO_2020 Grid should not be used for navi...
    history:      Information on the development of the data set and the sour...
    institution:  On behalf of the General Bathymetric Chart of the Oceans (G...
    node_offset:  1.0
    references:   DOI: 10.5285/836f016a-33be-6ddc-e053-6c86abc0788e
    source:       The GEBCO_2020 Grid is the latest global bathymetric produc...
    title:        The GEBCO_2020 Grid - a continuous terrain model for oceans...

In [20]:
import zarr

# Open the Zarr store
store = zarr.open_group('../data/GEBCO_2022_sub_ice_topo.zarr', mode='r')
print(list(store.keys()))
# Inspect attributes
for var in store.array_keys():
    arr = store[var]
    print(f"Variable: {var}")
    print(f"Compression: {arr.compressor}")
    print(f"Chunk Size: {arr.chunks}")

['crs', 'elevation', 'gebco', 'lat', 'lon']
Variable: crs
Compression: None
Chunk Size: ()
Variable: elevation
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (43200, 86400)
Variable: lat
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (21600,)
Variable: lon
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (43200,)


In [32]:
#import zarr

# Open the Zarr store
store2 = zarr.open_group('../data/GEBCO_2023_sub_ice_topo_compressor.zarr', mode='r')
print(list(store2.keys()))
## Inspect attributes
## Access the 'gebco' group
# gebco_group = store2['gebco']

## Inspect attributes within the 'gebco' group
#for var in gebco_group.array_keys():
#    arr = gebco_group[var]
#    print(f"Variable: {var}")
#    print(f"Compression: {arr.compressor}")
#    print(f"Chunk Size: {arr.chunks}")

for var in store2.array_keys():
    arr = store2[var]
    print(f"Variable: {var}")
    print(f"Compression: {arr.compressor}")
    print(f"Chunk Size: {arr.chunks}")

['elevation', 'lat', 'lon']
Variable: elevation
Compression: Zlib(level=1)
Chunk Size: (43200, 86400)
Variable: lat
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (21600,)
Variable: lon
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (43200,)


In [21]:
#import zarr

# Open the Zarr store
store = zarr.open_group('../data/GEBCO_2023_sub_ice_topo.zarr', mode='r')
print(list(store.keys()))
# Inspect attributes
for var in store.array_keys():
    arr = store[var]
    print(f"Variable: {var}")
    print(f"Compression: {arr.compressor}")
    print(f"Chunk Size: {arr.chunks}")

['elevation', 'lat', 'lon']
Variable: elevation
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (60, 60)
Variable: lat
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (21600,)
Variable: lon
Compression: Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Chunk Size: (43200,)


In [13]:
print(sys.getsizeof(dz))
dz['elevation'].nbytes/(1024 ** 3)

112


6.9522857666015625

In [14]:
st = time.time()
dz_s1 = dz.sel(lon=slice(105, 135), lat=slice(2, 35)) 
dz_s1.close() #try if it can close
zt1 = dz_s1.isel(lon=slice(1000, 1010), lat=slice(2000, 2010))
xz1 = zt1['elevation'].values
et = time.time()
print('Exe time: ', et-st, 'sec')
print(xz1)

Exe time:  40.998939990997314 sec
[[-121 -119 -116 -111 -106 -101  -94  -88  -82  -76]
 [-123 -121 -119 -115 -109 -103  -97  -91  -86  -79]
 [-126 -124 -122 -118 -113 -106 -100  -95  -88  -83]
 [-127 -127 -126 -123 -116 -110 -103  -97  -91  -86]
 [-127 -127 -128 -126 -118 -112 -106  -99  -94  -88]
 [-127 -127 -126 -125 -119 -113 -107 -102  -96  -91]
 [-127 -127 -126 -123 -118 -113 -108 -103  -99  -94]
 [-126 -125 -124 -122 -118 -113 -109 -104 -100  -96]
 [-125 -124 -123 -120 -118 -114 -110 -107 -102  -98]
 [-123 -123 -121 -120 -118 -114 -112 -108 -104 -100]]


In [ ]:
st = time.time()
jz1 = orjson.dumps(xz1.tolist(), option=orjson.OPT_NAIVE_UTC | orjson.OPT_SERIALIZE_NUMPY)
et = time.time()
print('4.1z Convert JSON by orjson time: ', et-st, 'sec')
print(jz1)
